### Marketing analysis
Import all necessary dependences

In [1]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.compose import ColumnTransformer

import category_encoders as ce

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, KernelPCA

from scipy import stats
import statsmodels.stats.power as power

from imblearn.pipeline import Pipeline as Pipe
from itertools import combinations
from imblearn.combine import SMOTETomek

from lightgbm import LGBMClassifier
import optuna
from optuna.samplers import TPESampler

import category_encoders as ce
import shap
!pip install -q BorutaShap
from BorutaShap import BorutaShap

seed = 123
np.random.seed(seed)
clear_output()

In [2]:
path = r'../input/customer-personality-analysis/marketing_campaign.csv'
TARGET_NAME = 'Response'

In [3]:
df = pd.read_csv(path, sep='\t')
df.sample(5).transpose()

In [4]:
def show_proba_calibration_plots(y_predicted_probs, y_true_labels):
    preds_with_true_labels = np.array(list(zip(y_predicted_probs, y_true_labels)))

    thresholds = []
    precisions = []
    recalls = []
    f1_scores = []

    for threshold in np.linspace(0.1, 0.9, 9):
        thresholds.append(threshold)
        precisions.append(precision_score(y_true_labels, list(map(int, y_predicted_probs > threshold))))
        recalls.append(recall_score(y_true_labels, list(map(int, y_predicted_probs > threshold))))
        f1_scores.append(f1_score(y_true_labels, list(map(int, y_predicted_probs > threshold))))

    scores_table = pd.DataFrame({'f1':f1_scores,
                                 'precision':precisions,
                                 'recall':recalls,
                                 'probability':thresholds}).sort_values('f1', ascending=False).round(3)
  
    figure = plt.figure(figsize = (15, 5))

    plt1 = figure.add_subplot(121)
    plt1.plot(thresholds, precisions, label='Precision', linewidth=4)
    plt1.plot(thresholds, recalls, label='Recall', linewidth=4)
    plt1.plot(thresholds, f1_scores, label='F1', linewidth=4)
    plt1.set_ylabel('Scores')
    plt1.set_xlabel('Probability threshold')
    plt1.set_title('Probabilities threshold calibration')
    plt1.legend(bbox_to_anchor=(0.25, 0.25))   
    plt1.table(cellText = scores_table.values,
               colLabels = scores_table.columns, 
               colLoc = 'center', cellLoc = 'center', loc = 'bottom', bbox = [0, -1.3, 1, 1])
    plt2 = figure.add_subplot(122)
    plt2.hist(preds_with_true_labels[preds_with_true_labels[:, 1] == 0][:, 0], 
              label='Another class', color='royalblue', alpha=1)
    plt2.hist(preds_with_true_labels[preds_with_true_labels[:, 1] == 1][:, 0], 
              label='Main class', color='darkcyan', alpha=0.8)
    plt2.set_ylabel('Number of examples')
    plt2.set_xlabel('Probabilities')
    plt2.set_title('Probability histogram')
    plt2.legend(bbox_to_anchor=(1, 1))

    plt.show()

def report(y_train, y_train_pred, y_test, y_test_pred, y_train_proba=None, y_test_proba=None):
    print('Train\n', classification_report(y_train, y_train_pred, digits=3))
    print('Test\n', classification_report(y_test, y_test_pred, digits=3))
    if y_train_proba is not None and y_test_proba is not None:
        roc_train, roc_test = roc_auc_score(y_train, y_train_proba), roc_auc_score(y_test, y_test_proba)
        print(f'Train ROC_AUC: {roc_train:.3f}, Test ROC_AUC: {roc_test:.3f}')
    print('Confusion Matrix', '\n', pd.crosstab(y_test, y_test_pred))

def reduce_memory(df, verbose=0):
    if verbose != 0:
        start_mem = df.memory_usage().sum() / 1024 ** 2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object and str(col_type)[:4] != 'uint' and str(col_type) != 'category':
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif str(col_type)[:4] != 'uint':
            df[col] = df[col].astype('category')
    if verbose != 0:
        end_mem = df.memory_usage().sum() / 1024 ** 2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

def roc_plot(y_true, probs):
    fpr, tpr, threshold = roc_curve(y_true, probs)
    roc_auc = auc(fpr, tpr)
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

def vote(predictions: list, weights: list):
    predictions = np.asarray(predictions).T
    maj_vote = np.apply_along_axis(lambda x: np.argmax(np.bincount(x, weights=weights)), axis=1, arr=predictions)
    return maj_vote

def cross_validation(clf, X, y, scoring='f1'):
    scores = cross_val_score(estimator=clf, X=X, y=y, cv=10, scoring=scoring, n_jobs=-1)
    print(f'Меры правильности перекрекстной оценки: {scores}')
    print(f'Точность перекретсной оценки: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')
    return scores

def effect_size(factor_a, factor_b, cohen=True, desired_power=0.8, alpha=0.05):
    n1, n2 = len(factor_a), len(factor_b)
    s1, s2 = factor_a.std(ddof=1), factor_b.std(ddof=1)
    df = (s1 ** 2 / n1 + s2 ** 2 / n2) ** 2 / \
          ((s1 ** 2 / n1) ** 2 / (n1 - 1) + (s2 ** 2 / n2) ** 2 / (n2 - 1))
    if cohen:
        sigma_pooled = np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
        return np.abs(factor_a.mean() - factor_b.mean()) / sigma_pooled, df
    else:
        return power.tt_ind_solve_power(effect_size=None, nobs1=len(factor_a), alpha=alpha, power=desired_power,
                                        ratio=len(factor_b)/len(factor_a), alternative='two-sided'), df
    
def statistic_output(*columns, df=df, cat=False, target=TARGET_NAME, alpha=0.05, sample_size=0):
    data = df.copy()
    data.drop_duplicates(inplace=True)
    if sample_size == 0:
        sample_size = int(0.05 * len(data))
    if not cat:
        columns = data.drop(target, axis=1).select_dtypes(exclude=['category', 'object']).columns
        for column in columns:
            df_sampled = data[[column, target]].sample(sample_size, random_state=seed)
            factor_a = df_sampled.loc[df_sampled[target] == 0][column]   
            factor_b = df_sampled.loc[df_sampled[target] == 1][column]
            var_a, var_b = factor_a.var(), factor_b.var()   
            _, pvalue = stats.shapiro(df_sampled[column])
            if pvalue >= alpha:
                _, pvalue = stats.ttest_ind(factor_a, factor_b, equal_var=False)
                test = power.TTestIndPower()
                eff_size, deg_free = effect_size(factor_a, factor_b, cohen=False)
                pow = test.power(effect_size=eff_size, nobs1=len(factor_a), alpha=alpha, df=deg_free, 
                                 ratio=len(factor_b)/len(factor_a), alternative='two-sided')
            else:
                _, pvalue = stats.mannwhitneyu(factor_a, factor_b)
                pow, eff_size = None, None
            if pvalue < alpha:
                result = f'with effect_size = {eff_size:.4f} and ttest power {pow*100:.2f}%' if pow is not None else ''
                print(f'Factor "{column}" has statistical impact on target (var_a: {var_a:.2f}, var_b: {var_b:.2f}). {result}')
            else:
                print(f'Factor "{column}" does not affect target.')
    else:
        for column in columns:
            print(column)
            categories = data[column].unique().tolist()
            for pair in combinations(categories, r=2):
                a, b = pair
                if a != b:
                    try:
                        data_ = data.loc[data[column].isin(pair), ['ID', column, target]].sample(sample_size, random_state=seed)
                    except ValueError:
                        continue
                    table = data_.pivot_table(values='ID', index=column, columns=target, aggfunc='count')
                    try:
                        _, pvalue, _, _ = stats.chi2_contingency(table, correction=False)
                    except ValueError:
                        continue
                    if pvalue >= alpha:
                        print(f'Categories {a} and {b} can be united. P-value: {pvalue:.6f}')
                    else:
                        pass
                        #print(f'Categories {a} and {b} have different frequencies with target.')
                        
def categorical_stats(df=df, target=TARGET_NAME, alpha=0.05, sample_size=500):
    data = df.copy().sample(sample_size)
    columns_to_analize = data.select_dtypes(include=['category', 'object']).columns
    weak_list = []
    for factor in columns_to_analize:
        if factor == target:
            continue
        print(f'{factor}')
        table = pd.crosstab(data[factor], data[TARGET_NAME])
        p_value = stats.chi2_contingency(table, correction=False)[1]
        if p_value < alpha:
            print(f'Feature {factor} has statistical impact on target. P-value: {p_value:.6f}')
        else:
            weak_list.append(factor)
    if len(weak_list) > 0:
        print(f'Statistically weak categorical features: ', *weak_list)

In [5]:
df.info()
# no missing values

In [6]:
df.describe().transpose()
# max income = 666666 looks like outliers
# MntMeatProducts, MntWines - check if there are outliers
# Z_CostContact, Z_Revenue look like constant features

In [7]:
df.describe(include=['object'])
# Dt_Customer - we will transform it
# others: check if we can unite categories
# cardinality of others is low, we may apply one-hot encoding 

In [8]:
df[TARGET_NAME].value_counts(normalize=True)
# pretty high imbalance, we will apply class weights, sampling techniques

In [9]:
Z_Cost = df.loc[df['Z_CostContact'] != 3.0, 'Z_CostContact'].sum()
print(f"Z_CostContact is {'not' if Z_Cost > 0 else ''}constant feature")
Z_Rev = df.loc[df['Z_Revenue'] != 11.0, 'Z_Revenue'].sum()
print(f"Z_Revenue is {'not' if Z_Rev > 0 else ''}constant feature")
if Z_Cost == 0:
    df.drop('Z_CostContact', axis=1, inplace=True)
if Z_Rev == 0:
    df.drop('Z_Revenue', axis=1, inplace=True)

#### Visualizations

In [10]:
numerical_cols = df.drop(['ID'], axis=1).select_dtypes(include=[np.int64]).columns.tolist()
numerical_cols = [column for column in numerical_cols if not column[:3] in ['Acc', 'Res', 'Tee', 'Kid', 'Com']]

plt.figure(figsize=(22, 20))
for idx, column in enumerate(numerical_cols):
    plt.subplot(4, 4, idx + 1)
    dist = 'Normal Distribution' if stats.shapiro(df[column].sample(100))[1] > 0.05 else 'Not normal distribution'
    plt.title(f'{column}: {dist}')
    sns.histplot(data=df, x=column, hue=TARGET_NAME, bins=50, kde=True)
plt.subplots_adjust(hspace=0.4, wspace=0.4)
plt.show()
# almost all num and mnt features are skewed
# features look like they may have impact on target
# maybe we have to deal with outliers - later

In [11]:
categorical_cols = df.drop('Dt_Customer', axis=1).select_dtypes(include=['object']).columns

plt.figure(figsize=(15, 6))
for idx, column in enumerate(categorical_cols, 1):
    plt.subplot(1, 2, idx)
    plt.title(f'{column}')
    sns.countplot(x=column, hue=TARGET_NAME, data=df)
plt.subplots_adjust(hspace=0.4, wspace=0.4)
plt.show()
# maybe we can unite some categories - let's make statistical tests later

In [12]:
plt.figure(figsize=(18,20))
for idx, column in enumerate(numerical_cols, 1):
    plt.subplot(4, 4, idx)
    sns.boxplot(y=df[column], x=df[TARGET_NAME], data=df)
    plt.title(f'{column}')
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.show()
# some features may appear as not important (medians are equal in different classes)
# we have some outliers in here: drop (if there are few) or drop only significant (IsolationForest to find out wich ones)
# impute some significant outliers or let them stay. We will test on lgbm/logreg base model

In [13]:
# let's see if there are any meaningful correlations
plt.figure(figsize = (14,12))
corr_matrix = df.corr()
corr_matrix = np.round(corr_matrix, 2)
corr_matrix[np.abs(corr_matrix) < 0.3] = 0
sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='coolwarm')
plt.title('Correlation matrix')
plt.show()
# no extra high correlations, except meat products and number of catalog purchases
# lot's of medium high correlations between income and amount of money spent on different goods categories
# let the BestSet class decide if there is a need of dropping some features

#### Some early obvious transforms

In [14]:
# make Age feature out of year of birth
df['Age'] = 2022 - df['Year_Birth']
df.drop('Year_Birth', axis=1, inplace=True)

In [15]:
# transform date into number of days
df["Dt_Customer"] = pd.to_datetime('02-04-2022') - pd.to_datetime(df["Dt_Customer"])
df["Dt_Customer"] = df["Dt_Customer"].dt.days

In [16]:
df = df[df['Age'] < 90]
df = df[df['Income'] < 600000]
print(f'After removing outliers: {len(df)}')
# about removing other possible outliers we will decide later

In [17]:
df.duplicated().sum()
# no duplicates

#### Some statistic tests

In [18]:
statistic_output(df=df.drop('Complain', axis=1), sample_size=200)
# as we can see lots of features are not important

In [19]:
statistic_output(*categorical_cols, cat=True, sample_size=200)
# features have small impact 

In [20]:
df['Marital_Status'].value_counts()

In [24]:
categorical_stats(sample_size=220)
# actually they are both weak
# unite married and together, others as single
# Family = together(or)single + children + teens

In [25]:
# unite categories
df['Marital_Status'] = df["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", 
                                                     "Absurd":"Single", "Widow":"Single", "YOLO":"Single", 
                                                     "Divorced":"Single", "Single":"Single", "Alone": "Single"})
df['Marital_Status'].value_counts()

In [26]:
df['Marital_Status'] = df['Marital_Status'].map({'Partner': 1, 'Single': 0})

In [27]:
# one-hot
df = pd.get_dummies(df, prefix=['Education'])

### Best split

In [28]:
base_lr = Pipeline(steps=[('scaler', MinMaxScaler()),
                          ('imputer', IterativeImputer(n_nearest_features=20, random_state=seed)),
                          ('lr', LogisticRegression(class_weight='balanced', random_state=seed))])
base_lgbm = LGBMClassifier(verbose=-1, is_unbalance=True)

In [29]:
X, y = df.drop(TARGET_NAME, axis=1), df[TARGET_NAME]

train_sizes, train_scores, test_scores = learning_curve(estimator=base_lr, X=X, y=y, 
                                                        train_sizes=np.linspace(0.1, 1.0, 10), cv=10, scoring='roc_auc', n_jobs=-1)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
plt.figure(figsize=(10, 8))
plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='правильность при обучении')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', linestyle='--', marker='s', markersize=5, 
         label='правильность при проверке')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.axvline(X.shape[0]*0.7, color='orange', linestyle='-.', label='test_size=0.3')
plt.grid()
plt.xlabel('Количество обучающих образцов')
plt.ylabel('Правильность')
plt.legend(loc='best')
plt.show()
# setting test_size=0.3 looks fine, but in case of validation data we will set
# test_size=0.25, valid_size=0.1

In [30]:
X, y = df.drop(TARGET_NAME, axis=1), df[TARGET_NAME]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, stratify=y, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, stratify=y_train, random_state=123)
print(f'Train size: {X_train.shape[0]}, Validation size: {X_valid.shape[0]}, Test size: {X_test.shape[0]}')

#### Outliers

In [31]:
detector = IsolationForest(n_estimators=100, contamination=0.005, n_jobs=-1, random_state=seed)
out_cols = [column for column in X.columns if column.startswith('Num') or column.startswith('Mnt')]
outliers = detector.fit_predict(X[out_cols])
out_index = X.loc[outliers == -1, out_cols].index
X.loc[outliers == -1, out_cols]

In [32]:
# raw predictions
base_lgbm.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False, early_stopping_rounds=5, eval_metric='roc_auc')
base_raw_train = base_lgbm.predict(X_train)
base_raw_test = base_lgbm.predict(X_test)
base_train_proba = base_lgbm.predict_proba(X_train)[:,1]
base_test_proba = base_lgbm.predict_proba(X_test)[:,1]

report(y_train, base_raw_train, y_test, base_raw_test, base_train_proba, base_test_proba)

base_lr.fit(X_train, y_train)
base_raw_train = base_lr.predict(X_train)
base_raw_test = base_lr.predict(X_test)
base_train_proba = base_lr.predict_proba(X_train)[:,1]
base_test_proba = base_lr.predict_proba(X_test)[:,1]

report(y_train, base_raw_train, y_test, base_raw_test, base_train_proba, base_test_proba)

#### Dropping possible outliers, IQR adjusting and imputing is removed as no significant improve was acheaved

### Feature engeneering, selecting and outlier classes

In [33]:
class FeatureCompose(BaseEstimator, TransformerMixin):
    def __init__(self, weights=np.ones(6)):
        self.weights = weights
        self.weight_vec = pd.Series(data=self.weights, index=['MntWines', 'MntFruits', 'MntMeatProducts',
                                                             'MntFishProducts', 'MntSweetProducts', 'MntGoldProds'],
                                   name='weights_vector')
        self.age_bin = None
        self.age = None
        self.median_income = None
        self.pipe = None
        self.cols = None
        
    def fit(self, X, y=None):
        X_ = X.dropna().copy()
        #X_['Age'] = 2022 - X_['Year_Birth']
        self.age = X_.groupby(by='Age')['Income'].agg('median').to_dict()
        self.median_income = X_['Income'].median()
        self.pipe = Pipeline(steps=[('scaler', MinMaxScaler()),
                                    ('cluster', KMeans(n_clusters=2, n_init=10, max_iter=300, random_state=seed, tol=1e-04))])
        self.cols = [column for column in X_.columns if column.startswith('Num') or column.startswith('Mnt') or column in ('Income', 'Recency')]
        self.pipe.fit(X_[self.cols])
        del X_
        return self
    
    def transform(self, X):
        # here it is not supposed to deal with 'isoforest' outliers
        X_ = X.copy()
        # Family size
        X_['Family_size'] = X_['Marital_Status']+1 + X_['Kidhome'] + X_['Teenhome']
        # Number of children
        X_['Num_child'] = X_['Kidhome'] + X_['Teenhome']
        # total number of purchases
        num_cols = [col for col in X_.columns if col.startswith('Num')]
        X_['Total_Purch'] = X_[num_cols].sum(axis=1)
        # total amount of spent money, weighted
        mnt_cols = [col for col in X_.columns if col.startswith('Mnt')]
        X_['Total_Mnt'] = (X_[mnt_cols] * self.weight_vec).sum(axis=1)
        # cuts
        X_['Med_age_income'] = X_['Age'].map(self.age)
        X_['Med_age_income'].fillna(self.median_income, inplace=True)
        # cluster distances
        X_[['cluster_0', 'cluster_1']] = self.pipe.transform(X_[self.cols])
        X_ = reduce_memory(X_)
        
        return X_
    
class DataFrameR(BaseEstimator, TransformerMixin):
    def __init__(self, columns, index=None):
        self.columns = columns
        self.index = index

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        assert len(self.columns) == X.shape[1], f'Number of columns {len(self.columns)} is not equal to data shape {X.shape[1]}'
        if self.index is not None:
            X_ = pd.DataFrame(X, columns=self.columns, index=self.index)
        else:
            X_ = pd.DataFrame(X, columns=self.columns)
        return X_
    
class RemoveOutliers(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        """Drop possible outliers to further apply IterativeImputer"""
        self.columns = columns
        self.detector = IsolationForest(n_estimators=100, contamination=0.005, n_jobs=-1, random_state=seed)
        
    def fit(self, X, y=None):
        out_cols = [column for column in X.columns if column.startswith('Num') or column.startswith('Mnt')]
        self.detector.fit(X[self.columns])
        return self
    
    def transform(self, X):
        outliers = self.detector.predict(X[self.columns])
        out_index = X[outliers == -1].index
        X.loc[out_index, self.columns] = None
        return X
    
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            return X
        return X[self.columns]

class BestSet(BaseEstimator, TransformerMixin):
    def __init__(self, estimator, k_features=12, scoring='f1', test_size=0.2):
        self.scoring = scoring
        self.k_features = k_features
        self.test_size = test_size
        self.estimator = clone(estimator)
        self.fit_params = {}
        if self.estimator.__class__.__name__ == 'LGBMClassifier':
            self.fit_params.update({'verbose': False})

    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
            X, y = X.values, y.values
        X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y, 
                                                            test_size=self.test_size, 
                                                            stratify=y,
                                                            random_state=seed)
        dim = X_train.shape[1]
        self.indices_ = tuple(range(dim))
        self.subsets_ = [self.indices_]
        score = self._calc_score(X_train, y_train, X_test, y_test, self.indices_)
        self.scores_ = [score]

        while dim > self.k_features:
            scores, subsets = [], []
            for p in combinations(self.indices_, r=dim-1):
                score = self._calc_score(X_train, y_train, X_test, y_test, p)
                scores.append(score)
                subsets.append(p)
            best = np.argmax(scores)
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1
            self.scores_.append(scores[best])
        self.k_score_ = self.scores_[-1]
        return self
    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        best_indices = self.subsets_[np.argmax(self.scores_)]
        return X[:, best_indices]

    def _calc_score(self, X_train, y_train, X_test, y_test, indices):
        self.estimator.fit(X_train[:, indices], y_train, **self.fit_params)
        if self.scoring == 'auc':
            y_pred = self.estimator.predict_proba(X_test[:, indices])[:,1]
            score = roc_auc_score(y_test, y_pred)
        else:
            y_pred = self.estimator.predict(X_test[:, indices])
            score = f1_score(y_test, y_pred)
        return score

In [34]:
end_columns = ['Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer',
       'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts',
       'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
       'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
       'Complain', 'Age', 'Education_2n Cycle', 'Education_Basic',
       'Education_Graduation', 'Education_Master', 'Education_PhD',
       'Family_size', 'Num_child', 'Total_Purch', 'Total_Mnt',
       'Med_age_income', 'cluster_0', 'cluster_1']

### Construct all from the beginning
Next we make simple preprocessing steps, split data and define base pipeline

In [84]:
df = pd.read_csv(path, sep='\t')
df.set_index('ID', inplace=True)
"""Common simple transforms"""
# drop constant
df.drop('Z_CostContact', axis=1, inplace=True)
df.drop('Z_Revenue', axis=1, inplace=True)
# Age
df['Age'] = 2022 - df['Year_Birth']
df.drop('Year_Birth', axis=1, inplace=True)
# days client on 02.04.2022
df["Dt_Customer"] = pd.to_datetime('02-04-2022') - pd.to_datetime(df["Dt_Customer"])
df["Dt_Customer"] = df["Dt_Customer"].dt.days
# outliers
df = df[df['Age'] < 90]
df = df[df['Income'] < 600000]
# status
df['Marital_Status'] = df["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", 
                                                     "Absurd":"Single", "Widow":"Single", "YOLO":"Single", 
                                                     "Divorced":"Single", "Single":"Single", "Alone": "Single"})
# map status
df['Marital_Status'] = df['Marital_Status'].map({'Partner': 1, 'Single': 0})
# Education one-hot
df = pd.get_dummies(df, prefix=['Education'])

"""Split"""
X, y = df.drop(TARGET_NAME, axis=1), df[TARGET_NAME]
# columns to drop outliers in pipeline
out_cols = [column for column in df.columns if column.startswith('Num') or column.startswith('Mnt')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, stratify=y, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, shuffle=True, stratify=y_train, random_state=123)
print(f'Train size: {X_train.shape[0]}, Validation size: {X_valid.shape[0]}, Test size: {X_test.shape[0]}')
# preserve indexes?

In [36]:
columns_to_scale = ['Income', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts',
                    'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'Kidhome', 'Teenhome',
                    'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
                    'NumStorePurchases', 'NumWebVisitsMonth', 'Age', 'Med_age_income', 'Family_size', 
                    'Num_child', 'Total_Purch', 'Total_Mnt']
columns_mnt_pca = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
columns_num_pca = ['NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']
not_pca = [column for column in end_columns if column not in columns_mnt_pca and column not in columns_num_pca]
not_scale = [column for column in end_columns if column not in columns_to_scale]

In [85]:
N = 0 # number of components to drop in PCA
scaler = Pipeline(steps=[('selector1', ColumnSelector(columns_to_scale)), 
                         ('scaler', MinMaxScaler()), 
                         ('df', DataFrameR(columns_to_scale))])
not_scaler = Pipeline(steps=[('selector2', ColumnSelector(columns=not_scale))])

union = FeatureUnion(transformer_list=[('scale', scaler),
                                       ('not_scale', not_scaler)])

starter_pipe = Pipeline(steps=[('preprocessor', FeatureCompose()),
                               ('union_scaling', union),
                               ('dframer', DataFrameR(columns_to_scale+not_scale)),                     
                              ])
pca_mnt = Pipeline(steps=[('selector_mnt', ColumnSelector(columns_mnt_pca)),
                          ('pca_mnt', PCA(n_components=len(columns_mnt_pca)-N, random_state=seed)),
                          ('framer_mnt', DataFrameR([f"mnt_{i+1}" for i in range(len(columns_mnt_pca)-N)]))])

pca_num = Pipeline(steps=[('selector_num', ColumnSelector(columns_num_pca)),
                          ('pca_mnt', PCA(n_components=len(columns_num_pca)-N, random_state=seed)),
                          ('framer_mnt', DataFrameR([f"num_{i+1}" for i in range(len(columns_num_pca)-N)]))])

union_2 = FeatureUnion(transformer_list=[('select_no_pca', ColumnSelector(not_pca)), 
                                         ('pca_MNT', pca_mnt),
                                         ('pca_NUM', pca_num)])

estimator_boost = LGBMClassifier(verbose=-1, is_unbalance=True, max_depth=3)
estimator = LogisticRegression(class_weight='balanced', random_state=seed, C=10.0)

pipe = Pipeline(steps=[('starter', starter_pipe),
                       ('pca', KernelPCA(n_components=len(end_columns)-7, random_state=seed)),
                       #('union_', union_2), # no improvement
                       #('feature_selector', BestSet(estimator=estimator, k_features=25, scoring='auc')), # not helping
                       ])
pipe_boost = Pipe(steps=[('preprocessor', FeatureCompose()),
                         ('feature_selector', BestSet(estimator=estimator_boost, k_features=25, scoring='auc')),
                         ('smote', SMOTETomek(sampling_strategy=.8, random_state=seed)),
                         ('model', LGBMClassifier(verbose=-1, is_unbalance=True))
                        ])

In [ ]:
selector = BorutaShap(model=LGBMClassifier(verbose=-1, is_unbalance=True), importance_measure='shap', classification=True)
selector.fit(X_train, y_train, n_trials=50, sample=False, verbose=False)
# lots of features are decided as unimportant

```
9 attributes confirmed important: ['AcceptedCmp5', 'AcceptedCmp1', 'Recency', 'Marital_Status', 'NumStorePurchases', 'MntMeatProducts', 'Income', 'AcceptedCmp3', 'Dt_Customer']
25 attributes confirmed unimportant: ['Education_Graduation', 'NumCatalogPurchases', 'Education_PhD', 'Med_age_income', 'MntWines', 'NumDealsPurchases', 'Age', 'MntFruits', 'Complain', 'NumWebVisitsMonth', 'Education_2n Cycle', 'Teenhome', 'Family_size', 'AcceptedCmp2', 'Num_child', 'Total_Mnt', 'AcceptedCmp4', 'cluster_0', 'NumWebPurchases', 'MntGoldProds', 'Kidhome', 'MntFishProducts', 'Education_Basic', 'Education_Master', 'MntSweetProducts']
2 tentative attributes remains: ['cluster_1', 'Total_Purch']
```

In [ ]:
remain = ['AcceptedCmp5', 'AcceptedCmp1', 'Recency', 'Marital_Status', 'NumStorePurchases', 'MntMeatProducts', 
          'Income', 'AcceptedCmp3', 'Dt_Customer', 'cluster_1', 'Total_Purch']
#X_train = X_train[remain]
#X_valid = X_valid[remain]
#X_test = X_test[remain]

In [86]:
# for logistic regression
X_train = pipe.fit_transform(X_train, y_train)
X_test = pipe.transform(X_test)
smote = SMOTETomek(sampling_strategy=.6, random_state=seed)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [87]:
lr = LogisticRegression(class_weight='balanced', random_state=seed)
lr.fit(X_train, y_train)
pred_train = lr.predict(X_train)
pred_test = lr.predict(X_test)
pred_train_proba = lr.predict_proba(X_train)[:,1]
pred_test_proba = lr.predict_proba(X_test)[:,1]

report(y_train, pred_train, y_test, pred_test, pred_train_proba, pred_test_proba)
# with kernel-pca instead of Best set selection we got another results, the bias is even more higher, but FN & FP are better
# a lot of linear correlations that are present in data and noise were eliminated by kernel-pca transformation

### LightGBM model

In [ ]:
X_train, y_train = pipe_boost[:3].fit_resample(X_train, y_train)
X_valid = pipe_boost[:2].transform(X_valid)

In [40]:
model_params = {
                'objective': 'binary', # cross_entropy
                'n_estimators': 800,
                'n_jobs': -1,
                'is_unbalance': True,
                'random_state': 123
}
fit_params = {'early_stopping_rounds': 10,  
              'eval_set': [(X_valid, y_valid)],  
              'eval_metric': 'auc',
              'verbose': False
}

In [ ]:
def objective(trial):
    param_trials = {
                    'max_depth': trial.suggest_int('max_depth', 3, 8),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
                    'reg_alpha': trial.suggest_float('reg_alpha', 1., 5.),
                    'reg_lambda': trial.suggest_float('reg_lambda', 1., 5.),
                    'num_leaves': trial.suggest_int('num_leaves', 20, 265),
                    'subsample': trial.suggest_float('subsample', 0.3, 1.),
                    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.),
                    'max_bin': trial.suggest_int('max_bin', 30, 260),
                    'min_child_samples': trial.suggest_int('min_child_samples', 80, 260),
                    'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 0.1),
                    'boosting_type': trial.suggest_categorical('boosting_type', ['goss', 'gbdt']),
                    }
    param_trials.update(model_params)
    opt_model = LGBMClassifier(**param_trials)
    opt_model.fit(X_train, y_train, **fit_params)
    
    y_pred = opt_model.predict_proba(X_valid)[:,1]
    score = roc_auc_score(y_valid, y_pred)
    
    return score

In [ ]:
optuna.logging.set_verbosity(optuna.logging.FATAL)
study = optuna.create_study(sampler=TPESampler(seed=seed), direction="maximize")
study.optimize(objective, n_trials=350, timeout=6000)

print(f'Number of completed trials: {len(study.trials)}')
print('Best trial')
trial = study.best_trial
print(f'Best score: {trial.value}')
print('Best params')
for key, value in trial.params.items():
    print(f'{key}: {value}')

In [41]:
model_params = {
                'objective': 'binary', # cross_entropy
                'n_estimators': 800,
                'n_jobs': -1,
                'is_unbalance': True,
                'random_state': 123,
                'max_depth': 7,
                'learning_rate': 0.4327804963936754,
                'reg_alpha': 4.677471904510624,
                'reg_lambda': 4.046531932784275,
                'num_leaves': 177,
                'subsample': 0.8872682894784513,
                'colsample_bytree': 0.8791931580090088,
                'max_bin': 102,
                'min_child_samples': 236,
                'min_child_weight': 0.0663351899300188,
                'boosting_type': 'gbdt',
}

In [42]:
# get result probabilities on 10 folds
test_results, train_roc, test_pred, train_f1, train_acc = [], [], [], [], []
skf = StratifiedKFold(n_splits=10)
X, y = df.drop(TARGET_NAME, axis=1), df[TARGET_NAME]
X_fold, X_test, y_fold, y_test = train_test_split(X,y, test_size=0.2, shuffle=True, stratify=y, random_state=seed)

for train_index, valid_index in skf.split(X_fold, y_fold):
    X_train, X_valid = X_fold.iloc[train_index, :], X_fold.iloc[valid_index, :]
    y_train, y_valid = y_fold.iloc[train_index], y_fold.iloc[valid_index]

    pipe_boost = Pipe(steps=[('preprocessor', FeatureCompose()),
                             ('feature_selector', BestSet(estimator=estimator_boost, k_features=28, scoring='auc')),
                             #('Scaler_for_pca', StandardScaler()),
                             #('kernel', KernelPCA(n_components=len(end_columns)-5, kernel='rbf', random_state=seed)),
                             ('smote', SMOTETomek(sampling_strategy=.8, random_state=seed)),
                             ('model', LGBMClassifier(verbose=-1, **model_params))
                            ])
    X_train, y_train = pipe_boost[:3].fit_resample(X_train, y_train)
    X_valid = pipe_boost[:2].transform(X_valid)
    X_test_ = pipe_boost[:2].transform(X_test)
    
    fit_params = {'early_stopping_rounds': 4,  
                  'eval_set': [(X_valid, y_valid)],  
                  'eval_metric': 'auc',
                  'verbose': False
              }

    pipe_boost[-1].fit(X_train, y_train, **fit_params)
    test_labels = pipe_boost[-1].predict(X_test_)
    test_pred.append(test_labels)
    
    train_proba = pipe_boost[-1].predict_proba(X_train)[:,1]
    train_roc.append(roc_auc_score(y_train, train_proba))
    
    train_labels = pipe_boost[-1].predict(X_train)
    train_f1.append(f1_score(y_train, train_labels))
    train_acc.append(accuracy_score(y_train, train_labels))
    
    pred_test = pipe_boost[-1].predict_proba(X_test_)[:,1]
    test_results.append(pred_test)

final_test = np.array(test_results).mean(axis=0)
test_pred = vote(test_pred, weights=np.ones(10))

In [43]:
print(f'train mean accuracy: {np.array(train_acc).mean():.4f}, test accuracy: {accuracy_score(y_test, test_pred):.4f}')
print(f'train mean f1: {np.array(train_f1).mean():.4f}, test f1: {f1_score(y_test, test_pred):.4f}')
print(f'train mean auc: {np.array(train_roc).mean():.4f}, test auc: {roc_auc_score(y_test, final_test):.4f}')
# model overfitted - high variance

In [44]:
print(confusion_matrix(y_test, test_pred))

In [45]:
show_proba_calibration_plots(final_test, y_test)

In [46]:
roc_plot(y_test, final_test)

In [52]:
from sklearn.feature_selection import VarianceThreshold

var = VarianceThreshold(threshold=1000)
var_selected = var.fit_transform(X_)
var.get_feature_names_out()
print('High variance features:')
print('-'*30)
for x, vr in zip(X_.columns, var.get_support()):
    if vr:
        print(f'{x}')

In [53]:
import lightgbm

fig, ax = plt.subplots(1, 1, figsize=(8,8))
lightgbm.plot_importance(pipe_boost[-1], ax=ax)
ax.set_yticklabels(X_.columns[[4,3,0,22,13,7,2,23,17,15,14,12,10,25,11,1,18,16,5]][::-1])
plt.show()

#### with pca decomposition
```
train mean accuracy: 0.8884, test accuracy: 0.8533
train mean f1: 0.8773, test f1: 0.5806
train mean auc: 0.9565, test auc: 0.8842
```

In [ ]:
print(classification_report(y_test, test_pred, digits=3))

### Observe shap values

In [48]:
X_ = pipe_boost[0].fit_transform(X_fold)
X_ = X_.iloc[:, list(pipe_boost[1].subsets_[np.argmax(pipe_boost[1].scores_)])]

In [49]:
explainer = shap.TreeExplainer(pipe_boost[-1])

data = pd.Series(data=X_.iloc[np.random.randint(len(X_)), :].values, index=X_.columns.tolist())
# Calculate Shap values
shap_values = explainer.shap_values(data.values.reshape(1,-1))

In [50]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data)

In [51]:
shap.initjs()

shap_values = explainer.shap_values(X_)
shap.summary_plot(shap_values[1], X_)

### Some thoughts: Logistic Regeression with KernelPCA showed better results than stratified LGBM model. One reason maybe is that the number of samples is too low